In [3]:
import pandas as pd
import numpy as np

In [24]:
# titanic
import seaborn as sns

titanic = sns.load_dataset('titanic')

titanic["age_b"] = pd.cut(titanic["age"], [0, 18, 80])

titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_b
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,"(18, 80]"
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,"(18, 80]"
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,"(18, 80]"
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,"(18, 80]"
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,"(18, 80]"


### aggregated data with multiple columns as multiindex series

In [6]:
agg = titanic.groupby(["sex", "class"], observed=False)["survived"].aggregate("mean")
print(agg)

sex     class 
female  First     0.968085
        Second    0.921053
        Third     0.500000
male    First     0.368852
        Second    0.157407
        Third     0.135447
Name: survived, dtype: float64


In [7]:
# manual pivot via unstack which unstacks the last level of the multiindex
agg.unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


### aggregated data with multiple columns

In [15]:
agg = titanic.groupby(["sex", "class"], observed=False).agg(
    count= ("survived", "count"),
    survived_pct= ("survived", "mean")
)
print(agg)

               count  survived_pct
sex    class                      
female First      94      0.968085
       Second     76      0.921053
       Third     144      0.500000
male   First     122      0.368852
       Second    108      0.157407
       Third     347      0.135447


In [23]:
# pivot
agg.reset_index().pivot(index="sex", columns="class", values="survived_pct")

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


### aggregated data with >2 columns

In [25]:
agg = titanic.groupby(["sex", "class","age_b"], observed=False).agg(
    count= ("survived", "count"),
    survived_pct= ("survived", "mean")
)
print(agg)

                        count  survived_pct
sex    class  age_b                        
female First  (0, 18]      11      0.909091
              (18, 80]     74      0.972973
       Second (0, 18]      14      1.000000
              (18, 80]     60      0.900000
       Third  (0, 18]      43      0.511628
              (18, 80]     59      0.423729
male   First  (0, 18]       5      0.800000
              (18, 80]     96      0.375000
       Second (0, 18]      15      0.600000
              (18, 80]     84      0.071429
       Third  (0, 18]      51      0.215686
              (18, 80]    202      0.133663


In [28]:
# pivot
agg.reset_index(level=2).pivot(columns="age_b", values="survived_pct")

age_b           (0, 18]  (18, 80]
sex    class                     
female First   0.909091  0.972973
       Second  1.000000  0.900000
       Third   0.511628  0.423729
male   First   0.800000  0.375000
       Second  0.600000  0.071429
       Third   0.215686  0.133663